In [1]:
import cudf
import numpy as np
import pandas as pd

%matplotlib inline
%load_ext autotime

In [2]:
features = [
    ('srcip', 'str')
    , ('sport', 'str')
    , ('dstip', 'str')
    , ('dsport', 'str')
    , ('proto', 'str')
    , ('state', 'str')
    , ('dur', 'float64')
    , ('sbytes', 'int64')
    , ('dbytes', 'int64')
    , ('sttl', 'int64')
    , ('dttl', 'int64')
    , ('sloss', 'int64')
    , ('dloss', 'int64')
    , ('service', 'str')
    , ('sload', 'float64')
    , ('dload', 'float64')
    , ('spkts', 'int64')
    , ('dpkts', 'int64')
    , ('swin', 'int64')
    , ('dwin', 'int64')
    , ('stcpb', 'int64')
    , ('dtcpb', 'int64')
    , ('smeansz', 'int64')
    , ('dmeansz', 'int64')
    , ('trans_depth', 'int64')
    , ('res_bdy_len', 'int64')
    , ('sjit', 'float64')
    , ('djit', 'float64')
    , ('stime', 'int64')
    , ('ltime', 'int64')
    , ('sintpkt', 'float64')
    , ('dintpkt', 'float64')
    , ('tcprtt', 'float64')
    , ('synack', 'float64')
    , ('ackdat', 'float64')
    , ('is_sm_ips_ports', 'int64')
    , ('ct_state_ttl', 'int64')
    , ('ct_flw_http_mthd', 'int64')
    , ('is_ftp_login', 'int64')
    , ('ct_ftp_cmd', 'int64')
    , ('ct_srv_src', 'int64')
    , ('ct_srv_dst', 'int64')
    , ('ct_dst_ltm', 'int64')
    , ('ct_src_ ltm', 'int64')
    , ('ct_src_dport_ltm', 'int64')
    , ('ct_dst_sport_ltm', 'int64')
    , ('ct_dst_src_ltm', 'int64')
    , ('attack_cat', 'str')
    , ('label', 'int64')
]

names  = [e[0] for e in features]
dtypes = [e[1] for e in features]

temp = []
temp.append(cudf.read_csv('../data/UNSW-NB15_1.csv', names=names, dtype=dtypes))
temp.append(cudf.read_csv('../data/UNSW-NB15_2.csv', names=names, dtype=dtypes))
temp.append(cudf.read_csv('../data/UNSW-NB15_3.csv', names=names, dtype=dtypes))
temp.append(cudf.read_csv('../data/UNSW-NB15_4.csv', names=names, dtype=dtypes))

df = cudf.concat(temp)

for data in temp:
    del data

temp = []

time: 1.44 s


In [3]:
to_keep = [
      'dur', 'proto', 'service', 'state', 'spkts', 'dpkts', 'sbytes', 'dbytes'
    , 'sttl', 'dttl', 'sload', 'dload', 'sloss', 'dloss'
    , 'sjit', 'djit', 'swin', 'stcpb', 'dtcpb', 'dwin', 'tcprtt', 'synack', 'ackdat'
    , 'ct_srv_src', 'ct_state_ttl'
    , 'ct_src_dport_ltm', 'ct_dst_sport_ltm', 'ct_dst_src_ltm'
    , 'ct_srv_dst'
    , 'label']

df = df[to_keep].reset_index(drop=True).reset_index()
df = df.rename({'index': 'id'})
df['id'] = df['id'].astype('int32')

time: 47 ms


In [4]:
df.describe()

,id,dur,spkts,dpkts,sbytes,dbytes,sttl,dttl,sload,dload,...,tcprtt,synack,ackdat,ct_srv_src,ct_state_ttl,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,ct_srv_dst,label
count,2.540047e+06,2.540047e+06,2.540047e+06,2.540047e+06,2.540047e+06,2.540047e+06,2.540047e+06,2.540047e+06,2.540047e+06,2.540047e+06,...,2.540047e+06,2.540047e+06,2.540047e+06,2.540047e+06,2.540047e+06,2.540047e+06,2.540047e+06,2.540047e+06,2.540047e+06,2.540047e+06
mean,1.270023e+06,6.587920e-01,3.328884e+01,4.272664e+01,4.339600e+03,3.642759e+04,6.278198e+01,3.076681e+01,3.695645e+07,2.450861e+06,...,6.180000e-03,3.288000e-03,2.893000e-03,9.206988e+00,2.611550e-01,4.642139e+00,3.592729e+00,6.845886e+00,8.988958e+00,1.264870e-01
std,7.332486e+05,1.392493e+01,7.628388e+01,1.215020e+02,5.640599e+04,1.610960e+05,7.462277e+01,4.285089e+01,1.186043e+08,4.224863e+06,...,4.615900e-02,2.593600e-02,2.394700e-02,1.083676e+01,6.830970e-01,8.477579e+00,6.174445e+00,1.125828e+01,1.082249e+01,3.323980e-01
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00
25%,6.350115e+05,1.037000e-03,2.000000e+00,2.000000e+00,2.000000e+02,1.780000e+02,3.100000e+01,2.900000e+01,1.353963e+05,1.191594e+04,...,0.000000e+00,0.000000e+00,0.000000e+00,2.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,2.000000e+00,0.000000e+00
50%,1.270023e+06,1.586100e-02,1.200000e+01,1.200000e+01,1.470000e+03,1.820000e+03,3.100000e+01,2.900000e+01,5.893038e+05,5.893179e+05,...,6.130000e-04,4.830000e-04,1.220000e-04,5.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,2.000000e+00,5.000000e+00,0.000000e+00
75%,1.905034e+06,2.145550e-01,4.400000e+01,4.200000e+01,3.182000e+03,1.489400e+04,3.100000e+01,2.900000e+01,2.039923e+06,2.925974e+06,...,7.020000e-04,5.540000e-04,1.400000e-04,1.000000e+01,0.000000e+00,2.000000e+00,1.000000e+00,5.000000e+00,1.000000e+01,0.000000e+00
max,2.540046e+06,8.786638e+03,1.064600e+04,1.101800e+04,1.435577e+07,1.465753e+07,2.550000e+02,2.540000e+02,5.988000e+09,1.287619e+08,...,1.003751e+01,4.525272e+00,5.512234e+00,6.700000e+01,6.000000e+00,6.700000e+01,6.000000e+01,6.700000e+01,6.700000e+01,1.000000e+00


time: 521 ms


In [5]:
bin_num = 10

quantiles = cudf.DataFrame()

for i in range(1, bin_num):
    quant = i / 10
    print(f'Quantile: {quant}')
    
    quantiles['q' + str(i)] = df.quantile(q=quant)

quantiles = quantiles.dropna().reset_index()
quantiles_list = quantiles.to_pandas().to_dict('records')

Quantile: 0.1
Quantile: 0.2
Quantile: 0.3
Quantile: 0.4
Quantile: 0.5
Quantile: 0.6
Quantile: 0.7
Quantile: 0.8
Quantile: 0.9
time: 2.55 s


In [6]:
qqq = []

for q in quantiles_list:
    if q['index'] not in ['id', 'label']:
        elements = list(q.items())
        quants = sorted(list(set([0.0] + [e[1] for e in elements[1:]])))
        qqq.append((elements[0][1], quants))

time: 908 µs


In [7]:
for q in qqq:
    col_name = q[0]
    bins = q[1]
    
    if df[col_name].dtype == 'int64':
        bins = [int(e) for e in bins]
        
    df[col_name + '_bin'] = df[col_name].digitize(np.array(bins))

time: 30.7 ms


In [8]:
quantized = [e[0] for e in qqq] + ['attack_cat', 'label']
df_binned = df[[e for e in list(df.columns) if e not in quantized] + ['label']]
del df

time: 19.3 ms


In [9]:
import cuml

encoders = {}
cols_to_encode = ['proto', 'service', 'state']

for col in cols_to_encode:
    le = cuml.preprocessing.LabelEncoder()
    df_binned[col] = df_binned[col].astype('category')
    df_binned[col + '_enc'] = le.fit_transform(df_binned[col])
    
    encoders[col] = le
    
# df_binned['dttl']
cols_reordered = [c for c in df_binned.columns if c not in cols_to_encode + ['label']] + ['label']
df_binned = df_binned[cols_reordered]

time: 642 ms


In [10]:
df_exploded = cudf.melt(df_binned, id_vars=['id', 'label']).sort_values('id').reset_index(drop=True)
df_exploded['variable'] = df_exploded['variable'].astype('str')
df_exploded['value'] = df_exploded['value'].astype('str')
df_exploded['feature'] = df_exploded['variable'] + '=' + df_exploded['value']
df_exploded = df_exploded[['id', 'feature', 'label']]
df_exploded.head()

,id,feature,label
0,0,dur_bin=3,0
1,0,spkts_bin=2,0
2,0,dpkts_bin=2,0
3,0,sbytes_bin=2,0
4,0,dbytes_bin=2,0


time: 606 ms


In [11]:
df_binned = df_binned[['id', 'label']]

time: 32.6 ms


# Encode features

In [12]:
features = df_exploded['feature'].unique().to_frame().reset_index()
features = features.rename({'index': 'feature_enc'})

df_exploded = df_exploded.merge(features, on='feature')
df_exploded.head()

,id,feature,label,feature_enc
0,605,ct_dst_sport_ltm_bin=2,0,6
1,605,ct_dst_src_ltm_bin=3,0,10
2,605,ct_srv_dst_bin=2,0,20
3,605,proto_enc=114,0,116
4,605,service_enc=4,0,250


time: 2.15 s


# Frequent items

In [13]:
import math

min_attack_rate = 0.75

def return_rank(frate, counts, rank):
    for i, (f, c) in enumerate(zip(frate, counts)):
        rank[i] = math.log(float(c)) * f

freq_items = df_exploded.groupby(['feature_enc']).agg({'id': 'count', 'label': 'sum'}).reset_index()
freq_items['attack_rate'] = freq_items['label'] / freq_items['id']

freq_items = freq_items.apply_rows(
    return_rank
    , incols = {'label': 'counts', 'attack_rate': 'frate'}
    , outcols = {'rank': np.float64}
    , kwargs = {}
) 

freq_items = freq_items.sort_values('rank', ascending=False)
freq_items.head(5)


#### TODO: apply min support
freq_items = freq_items.query(f'attack_rate >= {min_attack_rate} and label > 200')

time: 530 ms


# Find bicliques

In [14]:
max_iter = len(freq_items)
devices_checked = {}

patterns = []
stats    = []

features_ordered = list(freq_items['feature_enc'].to_array())

for i in range(max_iter):
    feature = features_ordered[i]
    
    ### get all the ids
    ids = df_exploded.query('feature_enc == @feature')['id'].unique().to_frame()
    h = ids.hash_columns(['id']).sum()
    
    print(f'Iter: {i}, Feature --> {feature}, devices analyzed --> {h in devices_checked}')
    
    if h not in devices_checked:    
        count_ids = len(ids)
        devices_checked[h] = 1
        
        ### OUTPUT PATTERN
        all_features = df_exploded.merge(ids, on='id').groupby('feature_enc').agg({'label': 'count'}).query('label == @count_ids').reset_index()
        all_features['pattern_id'] = i
        
        patterns.append(all_features[['pattern_id', 'feature_enc']])
        
        ### OUTPUT STATS
        ids = ids.merge(df_binned, on='id')
        ids['pattern_id'] = i
        ids = ids.groupby('pattern_id').agg({'id': 'count', 'label': 'sum'})
        ids = ids.rename({'id': 'packet_count', 'label': 'attack_count'})
        ids['attack_rate'] = ids['attack_count'] / ids['packet_count']
        ids['feature_cnt'] = all_features['feature_enc'].count()
        ids = ids.reset_index()
        stats.append(ids)

Iter: 0, Feature --> 38, devices analyzed --> False
Iter: 1, Feature --> 272, devices analyzed --> False
Iter: 2, Feature --> 253, devices analyzed --> False
Iter: 3, Feature --> 224, devices analyzed --> False
Iter: 4, Feature --> 113, devices analyzed --> False
Iter: 5, Feature --> 114, devices analyzed --> False
Iter: 6, Feature --> 213, devices analyzed --> False
Iter: 7, Feature --> 102, devices analyzed --> False
time: 2.85 s


In [15]:
patterns = cudf.concat(patterns).merge(features, on='feature_enc').sort_values(by='pattern_id')
stats = cudf.concat(stats)

time: 41 ms


In [16]:
stats

,pattern_id,packet_count,attack_count,attack_rate,feature_cnt
0,0,280466,262252,0.935058,1
0,1,230286,189097,0.821140,1
0,2,1533,1529,0.997391,6
0,3,274,274,1.000000,22
0,4,262,262,1.000000,19
0,5,262,262,1.000000,19
0,6,262,262,1.000000,19
0,7,260,260,1.000000,21


time: 18.6 ms


In [20]:
def to_set(x):
    return set(x)

def to_list(x):
    return list(x)

def ranges(bin_no, bins):
    if bin_no == len(bins):
        return '>{0:.0f}'.format(bins[-1])
    else:
        return '<{0:,.0f}, {1:,.0f})'.format(bins[bin_no-1], bins[bin_no])

quantile_bins = pd.DataFrame(qqq, columns=['feat', 'bins'])

patterns_host = patterns.to_pandas()
patterns_host['feat']    = patterns_host.apply(lambda row: row['feature'].split('='), axis = 1)
patterns_host['bin']     = patterns_host.apply(lambda row: row['feat'][1], axis = 1)
patterns_host['feat']    = patterns_host.apply(lambda row: row['feat'][0][:-4], axis = 1)
patterns_host            = patterns_host.merge(quantile_bins, on=['feat'])
patterns_host['ranges']  = patterns_host.apply(lambda row: ranges(int(row['bin']), row['bins']), axis=1)
patterns_host['feature'] = patterns_host['feat'] + '=' + patterns_host['ranges']
patterns_host            = patterns_host[['pattern_id', 'feature']].sort_values(by='pattern_id')

patterns_host_agg               = patterns_host.groupby('feature').agg({'pattern_id': to_list}).reset_index()
patterns_host_agg['pattern_id'] = patterns_host_agg.apply(lambda row: ','.join([str(e) for e in row['pattern_id']]), axis = 1)#
patterns_host_agg               = patterns_host_agg.groupby('pattern_id').agg({'feature': to_set}).reset_index()

patterns_host_agg.to_dict('records')

[{'pattern_id': '0,3,4,5,6,7', 'feature': {'ct_state_ttl=>2'}},
 {'pattern_id': '1', 'feature': {'sload=<11,417,080, 117,333,328)'}},
 {'pattern_id': '2',
  'feature': {'dttl=>29', 'dwin=>255', 'sttl=<60, 254)', 'swin=>255'}},
 {'pattern_id': '3', 'feature': {'sbytes=<146, 264)'}},
 {'pattern_id': '3,4,5,6,7',
  'feature': {'ackdat=<0, 0)',
   'dbytes=<0, 92)',
   'djit=<0, 0)',
   'dload=<0, 321)',
   'dloss=<0, 4)',
   'dpkts=<0, 2)',
   'dtcpb=<0, 638,417,164)',
   'dttl=<0, 29)',
   'dwin=<0, 255)',
   'sloss=<0, 3)',
   'stcpb=<0, 639,725,026)',
   'swin=<0, 255)',
   'synack=<0, 0)',
   'tcprtt=<0, 0)'}},
 {'pattern_id': '3,4,5,7', 'feature': {'sjit=<0, 1)'}},
 {'pattern_id': '3,6,7', 'feature': {'sttl=>254'}},
 {'pattern_id': '3,7', 'feature': {'spkts=<2, 4)'}}]

time: 138 ms
